In [ ]:
# configuration parameters for Times square
import datetime 
last_data_date =datetime.date.fromisoformat(datetime.datetime.now(datetime.UTC).strftime('%Y-%m-%d'))
duration_in_day = 1
OverViewOnly = True
max_time    =  100
fontsize    =  18

In [ ]:
from matplotlib import pyplot as plt

from lsst_efd_client import EfdClient,resample
import numpy as np
import pandas as pd
import datetime 
import pytz
timezone = pytz.utc
from math import isnan
import asyncio


In [ ]:
# initialisation from 
last_data_date_val = datetime.datetime.fromisoformat(last_data_date.isoformat()+'T14:00:00+00:00')
first_data_date_val = last_data_date_val - datetime.timedelta(days=duration_in_day)
#
tstart=first_data_date_val.isoformat()
tend=last_data_date_val.isoformat()
# save plot on file
save_fig=False
# show plot on screen 
show_it = True
#
day_ref=datetime.datetime.fromisoformat(tstart).replace(tzinfo=None)
print( 'we will analyse data between ',tstart,' and ',tend,' using FES configuration at ', day_ref)
run_cur='%s_%s' % (first_data_date_val.isoformat(),last_data_date_val.isoformat())
#
# ------------- initialisation 
# --- Manual configuration  ( to be edited by hand )  
#
#
output_data='/home/a/antilog/public_html/LsstCam/fes/'
#
# --- Automated configuration 
client = EfdClient('usdf_efd')
#
if save_fig  :
    print('Output will be saved in %s%s' % (output_data,run_cur))
#---------
manual=False
#
socket_name=[]
for i in range(5) : 
    socket_name.append('socket%d' % (i+1))
#


In [ ]:
#
def SaveFig(fig,File,run_cur='',raft_cur='',ccd_cur='',hdu=''):
    if len(hdu)==0 : 
        root_plt=os.path.join(output_data,run_cur,raft_cur,ccd_cur)
    else : 
        hdu_cur='%s' % (hdu)
        root_plt=os.path.join(output_data,run_cur,raft_cur,ccd_cur,hdu_cur)
    # 
    PlotFile=File.replace('.','_')
    PlotFile=PlotFile.replace(' ','_')
    os.makedirs(root_plt,exist_ok=True)
    plotfile=os.path.join(root_plt,PlotFile)
    print ('PlotFile=',plotfile)
    fig.savefig(plotfile,bbox_inches='tight')
    plt.close(fig) 
    return


In [ ]:
async def read_db(query,verbose=False,clean=True) :
    if verbose : print(query)
    db_in =  await client._influx_client.query(query)
    for col in db_in.columns :
        len_col=len(db_in[db_in[col].notnull()])
        if clean :
            if len_col==0 : 
                db_in.drop(columns=[col],inplace=True)
                txt='dropped'
            else :
                txt='kept'
        else :
            txt='kept'  
        if verbose : print('%s (%s) = %d ' % (col,txt,len_col) )
    return db_in 

In [ ]:
query = '''SELECT "mainDoorOpeningPercentage" FROM "efd"."autogen"."lsst.sal.ATDome.position" WHERE  time > '%s' AND time < '%s' ''' % (tstart,tend)
dome  =  await read_db(query,clean=False)
#
query = '''SELECT * FROM "lsst.MTCamera"."autogen"."lsst.MTCamera.fcs_duration_carousel_clamps" WHERE  time > '%s' AND time < '%s' AND "Agent"='fcs' AND "Hardware"='carousel' ''' % (tstart,tend)
cl_duration  =  await read_db(query,clean=False)
#
query = '''SELECT * FROM "lsst.MTCamera"."autogen"."lsst.MTCamera.fcs_duration" WHERE  time > '%s' AND time < '%s' AND "Agent"='fcs'  ''' % (tstart,tend)
set_duration  = await read_db(query,clean=False)
#
query = '''SELECT * FROM "lsst.MTCamera"."autogen"."lsst.MTCamera.fcs_counters" WHERE  time > '%s' AND time < '%s' AND "Agent"='fcs'  ''' % (tstart,tend)
set_counter  = await read_db(query,clean=False)
#
query = '''SELECT "profileVelocity" FROM "lsst.MTCamera"."autogen"."lsst.MTCamera.fcs_canbus_controller" WHERE  time > '%s' AND time < '%s'  AND "Controller"='carouselController' AND "Canbus"='canbus0' AND "Agent"='fcs' ''' % (tstart,tend)
orientation =  await read_db(query)
#
query = '''SELECT * FROM "lsst.MTCamera"."autogen"."lsst.MTCamera.fcs_canbus_controller" WHERE  time > '%s' AND time < '%s'  AND "Controller"='carouselController' AND "Canbus"='canbus0' AND "Agent"='fcs' ''' % (tstart,tend)
controler =  await read_db(query)
#
counter_allr=pd.merge_asof(cl_duration,set_counter[set_counter['SET_FILTER'].notnull()]['SET_FILTER'],left_index=True, right_index=True,suffixes=('_cl', ''), tolerance=pd.Timedelta("200s"),direction='backward' )
counter_all=pd.merge_asof(counter_allr,orientation[orientation['profileVelocity'].notnull()]['profileVelocity'],left_index=True, right_index=True,suffixes=('', ''), tolerance=pd.Timedelta("200s"),direction='forward' )
filter_dome=pd.merge_asof(set_duration[set_duration['SET_FILTER'].notnull()]['SET_FILTER'],dome[dome['mainDoorOpeningPercentage'].notnull()]['mainDoorOpeningPercentage'],left_index=True, right_index=True,suffixes=('_count', ''), tolerance=pd.Timedelta("100m"),direction='backward' )
duration_all=pd.merge_asof(counter_all,filter_dome,left_index=True, right_index=True,suffixes=('_count', ''), tolerance=pd.Timedelta("200s"),direction='forward' )


In [ ]:
brake=controler[(controler['brakeActivated'].notnull())]['brakeActivated'].to_numpy()
state=controler[(controler['brakeActivated'].notnull())]['state'].to_numpy()
delta_ft=pd.Timedelta("200s")
index_brakeOff_i=np.argwhere( (brake[0:-1]==True) & (state[1:]!=None)  & (brake[1:]==False ))[:,0]
start_controler=controler[(controler['brakeActivated'].notnull())].iloc[index_brakeOff_i].index
index_brakeOn_i=np.argwhere( (brake[0:-1]==False) & (state[0:-1]!=None)  & (brake[1:]==True ))[:,0]
end_controler=controler[(controler['brakeActivated'].notnull())].iloc[index_brakeOn_i+1].index
delta=end_controler-start_controler
break_warm=(controler.loc[start_controler]['positionSensorType'].to_numpy()==516)
deltas=np.array(delta.total_seconds())
#
deltap=[]
for i in range(len(index_brakeOff_i)) : 
    spos=controler[(controler['brakeActivated'].notnull())].iloc[index_brakeOff_i[i]:index_brakeOn_i[i]+1]
    pos=spos[(spos['positionSensorType']==520.0)]['position'].to_numpy()
    if (len(pos) == 0 ) :
        pos=spos[(spos['positionSensorType']==516.0)]['position'].to_numpy()
    deltap.append(pos[-1]-pos[0])
deltap=np.array(deltap)
#
deltax=np.zeros((len(start_controler)))
not_first=False
new=start_controler[0]
deltax[0]=-1
for i in range(len(start_controler)) : 
    old=new
    new=start_controler[i]
    if not_first :
        deltax[i]=(new-old).total_seconds()/3600
    not_first=True
#            
cur_in_on=[]
for i in range(len(start_controler)) : 
    cur_in_on.append(controler.loc[start_controler[i]:end_controler[i]]['averageCurrent'].to_numpy().mean())
cur_in_on=np.array(cur_in_on)
# all setfilter on sky 
full_set=duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Clamp']=='Xplus' ) & (duration_all['profileVelocity']>3000) & (duration_all['SET_FILTER'].notnull()) ]
# index with recovery due break unstick preventive operation 
break_warm_index=[]
for i in range(len(break_warm)) : 
    if break_warm[i] :
        #rint(i,start_controler[i],deltas[i],break_warm[i],cur_in_on[i])
        break_warm_index.append(start_controler[i])
idx=full_set.index.get_indexer(break_warm_index,method='bfill',tolerance=delta_ft)
un_stick=[]
for i in range(len(idx) ): 
   if idx[i] !=-1 :
        un_stick.append(idx[i])
# index with recovery due to unbalance
idy=full_set.index.get_indexer(start_controler,method='bfill',tolerance=delta_ft)
nb_break_on=0
un_balance=[]
last_set=-2
for i in range(len(idy) ):
    if idy[i]==-1  :
        # not no match  
        continue
    elif (idy[i] in idx) : 
        # not   break warm we skip it
        continue
    elif idy[i] == last_set and not(idy[i] in un_balance ) :
        # so this is an unbalance  recovery  
        un_balance.append(idy[i])
    else :
        last_set=idy[i]
# index with 0, 1, 2 socket rotation 
nb_sock=np.int32(np.abs(deltap)/0.87e6)
#for i in range(len(deltap)) : 
#    if abs(deltap[i]
#id2=full_set.index.get_indexer(start_controler,method='bfill',tolerance=delta_ft)     
        
    

In [ ]:
color_1_p=[]
color_1_l=[]=[]
color_1_g=[]
color_1=[]
color_2_p=[]
color_2_l=[]
color_2_g=[]
color_2=[]
for i in range(len(idy)) :
    if idy[i]==-1  : 
        continue        
    if nb_sock[i] == 1 :
       if (idy[i] in un_stick) and (idy[i] in un_balance) :
            color_1_p.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       elif (idy[i] in un_stick):
            color_1_l.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       elif (idy[i] in un_balance):
            color_1_g.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       else :
            color_1.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
    elif nb_sock[i] == 2 :
       if (idy[i] in un_stick) and (idy[i] in un_balance) :
            color_2_p.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       elif (idy[i] in un_stick):
            color_2_l.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       elif (idy[i] in un_balance):
            color_2_g.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
       else :
            color_2.append(full_set.loc[full_set.index[idy[i]]]['SET_FILTER']/1000)
to_plot=[color_1,color_1_g,color_1_l,color_1_p,color_2,color_2_g,color_2_l,color_2_p]
color  =['greenyellow'      ,'greenyellow'     ,'greenyellow'     ,'greenyellow'    ,'cyan'      ,'cyan'      ,'cyan'      ,'cyan' ]
hatch  =['.'      ,'\\'     ,'/'      ,'X'    ,'.'      ,'\\'     ,'/'      ,'X' ] 
label  =['1 Socket Rotation' ,'Unbalance fix','Break Unstick'  , 'Both' , '2 Sockets Rotation','Unbalance fix' ,'Break Unstick' , 'Both']
#

#
fig=plt.figure(figsize=(20,12))
for_mean=np.array(color_1_p+color_1_l+color_1_g+color_1+color_2_p+color_2_l+color_2_g+color_2)
all_value_mean=for_mean.mean()
all_value_nb=len(for_mean)
all_value_max=np.sum(for_mean>max_time)

title='SetFilter  duration  \n for a period from %s to %s \n  mean = %4.1f s for %d entries ' % (tstart,tend,all_value_mean,all_value_nb)
fig.suptitle(title,fontsize=fontsize)

a=plt.hist(to_plot,80,range=(75,max_time),stacked=True,hatch=hatch,color=color,label=label)
labelx='SetFilter Duration in s  ' 
if all_value_max > 0  : 
    labelx+=' ( %d SET_FILTER TOOK MORE THAN %ds ! ) ' % (all_value_max,max_time)
plt.xlabel(labelx,fontsize=fontsize) 
plt.legend()
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(fontsize=fontsize)

if show_it : plt.show()
if save_fig : SaveFig(fig,File='SetFilterOnSky',run_cur=run_cur,raft_cur='ClampOnCarousel')


In [ ]:
# Duration Plots 
color={'Xplus':['black','violet','b','purple','cyan'],'Xminus':['darkgreen','yellowgreen','lime','orange','gold']}
sym={'Xplus':'+','Xminus':'_'}
#
#
fig=plt.figure(figsize=(20,12))
title='SetFilter  duration \n for a period from %s to %s ' % (tstart,tend)
fig.suptitle(title,fontsize=fontsize)
a=[]
label=[]
all_value=np.array([])
for isock in range(1,6) : 
    for car_clamp in ['Xminus'] :
       val=duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']==car_clamp ) & (duration_all['profileVelocity']>3000 ) & (duration_all['SET_FILTER'].notnull()) ]['SET_FILTER'].to_numpy()/1000.
       val_mean=val[np.where((val>70) & (val<max_time))].mean()
       a.append(val)
       all_value=np.append(all_value,val) 
       val_nb=len(val[np.where((val>70) & (val<max_time))])
       label.append('%s mean = %4.1f for %d entries' % (socket_name[isock-1],val_mean,val_nb))
plt.hist(a,100,range=(70,max_time),stacked=True,color=color[car_clamp],label=label)
all_value_mean=all_value[np.where((all_value>70) & (all_value<max_time))].mean()
all_value_max_time=len(all_value[np.where(all_value>max_time)])
all_value_nb=len(all_value[np.where((all_value>70) & (all_value<max_time))])
labelx='SetFilter Duration in s ; mean[70s,%ds] = %4.1f s for %d entries ' % (max_time,all_value_mean,all_value_nb)
if all_value_max_time > 0  : 
    labelx+=' ( %d SET_FILTER TOOK MORE THAN %ds ! ) ' % (all_value_max_time,max_time)
plt.xlabel(labelx,fontsize=fontsize) 
plt.legend()
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(fontsize=fontsize)

if show_it : plt.show()
if save_fig : SaveFig(fig,File='SetFilterDurationHist',run_cur=run_cur,raft_cur='ClampOnCarousel')
#
fig=plt.figure(figsize=(20,12))
title='SetFilter  duration Dome Open \n for a period from %s to %s ' % (tstart,tend)
fig.suptitle(title,fontsize=fontsize)
a=[]
label=[]
all_value=np.array([])
for isock in range(1,6) : 
    for car_clamp in ['Xminus'] :
       val=duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']==car_clamp ) & (duration_all['profileVelocity']>3000 ) & (duration_all['SET_FILTER'].notnull())& (duration_all['mainDoorOpeningPercentage']>50 ) ]['SET_FILTER'].to_numpy()/1000.
       if len(val)==0 : 
           continue
       val_mean=val[np.where((val>70) & (val<max_time))].mean()
       a.append(val)
       all_value=np.append(all_value,val) 
       val_nb=len(val[np.where((val>70) & (val<max_time))])
       label.append('%s mean = %4.1f for %d entries' % (socket_name[isock-1],val_mean,val_nb))
if len(a) > 0 : 
    plt.hist(a,100,range=(70,max_time),stacked=True,color=color[car_clamp],label=label)
    all_value_mean=all_value[np.where((all_value>70) & (all_value<max_time))].mean()
    all_value_150=len(all_value[np.where(all_value>max_time)])
    all_value_nb=len(all_value[np.where((all_value>70) & (all_value<max_time))])
    labelx='SetFilter Duration Dome Open in s ; mean[70s,%ds] = %4.1f s for %d entries ' % (max_time,all_value_mean,all_value_nb)
    if all_value_150 > 0  : 
        labelx+=' ( %d SET_FILTER TOOK MORE THAN %ds ! ) ' % (all_value_150,max_time)
    plt.xlabel(labelx,fontsize=fontsize) 
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.legend(fontsize=fontsize)
    
    plt.legend()
    if show_it : plt.show()
    if save_fig : SaveFig(fig,File='SetFilterDurationHist',run_cur=run_cur,raft_cur='ClampOnCarousel')
#
fig=plt.figure(figsize=(20,12))
title='SetFilter  duration \n for a period from %s to %s ' % (tstart,tend)
fig.suptitle(title,fontsize=fontsize)
for isock in range(1,6) : 
        if len(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['profileVelocity']>3000) & (duration_all['SET_FILTER'].notnull()) & (duration_all['mainDoorOpeningPercentage']>50 ) ]) :
            marker='+'
            label='%s Dome Open' % (socket_name[isock-1])
            plt.plot(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['profileVelocity']>3000) & (duration_all['SET_FILTER'].notnull()) & (duration_all['mainDoorOpeningPercentage']>50 ) ]['SET_FILTER']/1000.,' ',marker=marker,color=color['Xplus'][isock-1],markersize=14,label=label)
        if len(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['profileVelocity']>3000)  & (duration_all['SET_FILTER'].notnull())& ( ~(duration_all['mainDoorOpeningPercentage'].notnull())  | (duration_all['mainDoorOpeningPercentage']<=50 ) )]['SET_FILTER']) > 0 :
            marker='_'
            label='%s Dome closed' % (socket_name[isock-1])
            plt.plot(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['profileVelocity']>3000)  & (duration_all['SET_FILTER'].notnull())& ( ~(duration_all['mainDoorOpeningPercentage'].notnull())  | (duration_all['mainDoorOpeningPercentage']<=50 ) )]['SET_FILTER']/1000.,' ',marker=marker,color=color['Xplus'][isock-1],markersize=14,label=label)
        if  len(duration_all[(duration_all['SET_FILTER'].notnull()) & (duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['SET_FILTER'].notnull()) & (duration_all['profileVelocity']<3000) ]['SET_FILTER']) > 0 :
            marker='^'
            label='%s horizon' % (socket_name[isock-1])
            plt.plot(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']=='Xplus' ) & (duration_all['SET_FILTER'].notnull()) & (duration_all['profileVelocity']<3000) ]['SET_FILTER']/1000.,' ',marker=marker,color=color['Xplus'][isock-1],markersize=14,label=label)
plt.xlabel('Date (UTC)',fontsize=fontsize)
plt.ylabel('SetFilter Duration in s',fontsize=fontsize) 
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.legend()
if show_it : plt.show()
if save_fig : SaveFig(fig,File='SetFilterDuration',run_cur=run_cur,raft_cur='ClampOnCarousel')
#
fig=plt.figure(figsize=(20,12))
title='SetFilter  count \n for a period from %s to %s ' % (tstart,tend)
fig.suptitle(title,fontsize=fontsize)
for isock in range(1,6) : 
    for car_clamp in ['Xplus'] :
        nb_sock=len(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']==car_clamp ) & (duration_all['SET_FILTER_count'].notnull())]['SET_FILTER_count'])
        label_count='%s %d change' % (socket_name[isock-1],nb_sock)
        plt.plot(duration_all[(duration_all['UNLOCK'].notnull()) & (duration_all['Socket']==socket_name[isock-1] )& (duration_all['Clamp']==car_clamp ) & (duration_all['SET_FILTER_count'].notnull())]['SET_FILTER_count'],' ',marker=sym[car_clamp],color=color[car_clamp][isock-1],markersize=14,label=label_count)
plt.xlabel('Date (UTC)',fontsize=fontsize)
plt.ylabel('SetFilter count',fontsize=fontsize) 
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(fontsize=fontsize)
plt.legend()
if show_it : plt.show()
if save_fig : SaveFig(fig,File='SetFilterCount',run_cur=run_cur,raft_cur='ClampOnCarousel')
#



In [ ]:
val

In [ ]:
# Delta time between filter change
time_start=duration_all[(duration_all['UNLOCK'].notnull())& (cl_duration['Clamp']=='Xplus') ]['SET_FILTER_count'].index
nb_change=len(time_start)
delta_change=time_start[1:]-time_start[0:-1]
min_TimeChange=delta_change.min().total_seconds()
min_allowed=90+120
nb_changem3s=np.sum(delta_change.total_seconds()<min_allowed)
min_allowed_h=(min_allowed)/3600.
#
if len(time_start)>1 :
    fig=plt.figure(figsize=(20,12))
    title='Time between filter changes \n Shorter time between filter change %f s , %d filter change within less than %d s   \n for a period from %s to %s ' % (min_TimeChange,nb_changem3s,min_allowed,tstart,tend)
    fig.suptitle(title,fontsize=fontsize)
    plt.plot(time_start[1:],delta_change.total_seconds()/3600,'+')
    plt.plot([time_start[1],time_start[-1]],[min_allowed_h,min_allowed_h],'r',label='ToO like speed boundary')
    plt.ylabel('Delta Time in hours',fontsize=fontsize)
    plt.xlabel('Date of the 2nd filter change',fontsize=fontsize)
    plt.yscale('log')
    plt.legend()
    if show_it  : plt.show()
    if save_fig : SaveFig(fig,File='SetFilterCount',run_cur=run_cur,raft_cur='ClampOnCarousel')
    fig=plt.figure(figsize=(20,12))
    title='Time between filter change \n Shorter time between filter change %f s , %d filter change within less than %d s   \n for a period from %s to %s ' % (min_TimeChange,nb_changem3s,min_allowed,tstart,tend)
    fig.suptitle(title,fontsize=fontsize)
    plt.hist(delta_change.total_seconds()/3600,100)
    #plt.plot([time_start[1],time_start[-1]],[min_allowed_h,min_allowed_h],'r')
    plt.xlabel('Delta Time in hours',fontsize=fontsize)
    plt.plot([min_allowed_h,min_allowed_h],[0,1],'r',label='ToO like speed boundary')
    plt.legend()   
    #plt.xscale('log')
    if show_it  : plt.show()
    if save_fig : SaveFig(fig,File='SetFilterCountHist',run_cur=run_cur,raft_cur='ClampOnCarousel')
    fig=plt.figure(figsize=(20,12))
    title='Time between filter change \n Shorter time between filter change %f s , %d filter change within less than %d s   \n for a period from %s to %s ' % (min_TimeChange,nb_changem3s,min_allowed,tstart,tend)
    fig.suptitle(title,fontsize=fontsize)
    plt.hist(delta_change.total_seconds(),1000)
    #plt.plot([time_start[1],time_start[-1]],[min_allowed_h,min_allowed_h],'r')
    plt.xlabel('Delta Time in seconds',fontsize=fontsize)
    plt.plot([min_allowed,min_allowed],[0,1],'r',label='ToO like speed boundary')
    
    plt.xscale('log')
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.legend(fontsize=fontsize)

    plt.legend()
    if show_it  : plt.show()
    if save_fig : SaveFig(fig,File='SetFilterCountHistSeconds',run_cur=run_cur,raft_cur='ClampOnCarousel')